In [63]:
import tensorflow as tf

In [64]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data/', one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [65]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

### 이전코드
```py
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)

L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
```

In [76]:
L1 = tf.layers.conv2d(X, 32, [3, 3])
L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2])
L1 = tf.layers.dropout(L1, 0.7, is_training)

### 이전코드
```py
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)

L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
```

In [77]:
L2 = tf.layers.conv2d(L1, 64, [3, 3])
L2 = tf.layers.max_pooling2d(L2, [2, 2], [2, 2])
L2 = tf.layers.dropout(L2, 0.7, is_training)

### 이전코드
```py
W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
```

In [78]:
L3 = tf.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation=tf.nn.relu)
L3 = tf.layers.dropout(L3, 0.7, is_training)

### 이전코드
```py
model = tf.matmul(L3, W4)
```

In [79]:
model = tf.layers.dense(L3, 10, activation=None)

In [80]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [81]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [82]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

In [83]:
for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost],
                                feed_dict={X: batch_xs, 
                                           Y: batch_ys,
                                           is_training: True})
        total_cost += cost_val
        
    print('Epoch :', '%04d' % (epoch + 1),
         'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))

print('finish!')

Epoch : 0001 Avg. cost = 0.170
Epoch : 0002 Avg. cost = 0.050
Epoch : 0003 Avg. cost = 0.031
Epoch : 0004 Avg. cost = 0.023
Epoch : 0005 Avg. cost = 0.017
Epoch : 0006 Avg. cost = 0.011
Epoch : 0007 Avg. cost = 0.010
Epoch : 0008 Avg. cost = 0.011
Epoch : 0009 Avg. cost = 0.008
Epoch : 0010 Avg. cost = 0.006
Epoch : 0011 Avg. cost = 0.007
Epoch : 0012 Avg. cost = 0.006
Epoch : 0013 Avg. cost = 0.009
Epoch : 0014 Avg. cost = 0.005
Epoch : 0015 Avg. cost = 0.004
finish!


In [85]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

print('accuracy:', sess.run(accuracy,
                           feed_dict={X: mnist.test.images.reshape(
                                                   -1, 28, 28, 1),
                                     Y: mnist.test.labels,
                                     is_training: False}))

accuracy: 0.9879
